## GOLD TO API LAYER

### API Layer


In [13]:
import json
import pandas as pd
from ETLTools import GlobalPath

In [14]:
# Define file paths using GlobalPath
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holdings/CurrentHoldings_data.csv"
)
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)
holdingshistory_gold_file_path = GlobalPath(
    "DATA/GOLD/Holdings/HoldingsHistory_data.csv"
)
profitloss_gold_file_path = GlobalPath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)
api_file_path = GlobalPath("DATA/API/API_data.json")

### Load and process holdings data from the GOLD layer


In [15]:
# Load holdings data from GOLD layer
df_current_holding = pd.read_csv(current_holding_records_file_path.path)
df_current_holding["datetime"] = pd.to_datetime(df_current_holding["datetime"])
print(
    f"Loaded GOLD Layer datetime data from: {current_holding_records_file_path.path}"
)

Loaded GOLD Layer datetime data from: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holdings\CurrentHoldings_data.csv


In [16]:
# Group by scrip_name, symbol, exchange, and segment to calculate totals and min datetime
df_grouped = (
    df_current_holding.groupby(["scrip_name", "symbol", "exchange", "segment"])
    .agg(
        total_quantity=("quantity", "sum"),
        total_amount=("amount", "sum"),
        min_datetime=("datetime", "min"),
    )
    .reset_index()
)

# Calculate average price
df_grouped["avg_price"] = (
    df_grouped["total_amount"] / df_grouped["total_quantity"]
)

### Load and process stock prices from the SILVER layer


In [17]:
# Load stock prices data from SILVER layer
df_stockprice = pd.read_csv(stockprice_silver_file_path.path)
print(
    f"Loaded SILVER Layer stock price data from: {stockprice_silver_file_path}"
)

# Convert the 'date' column to datetime type
df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])

# Extract the latest closing price for each symbol
df_stockprice["close_price"] = df_stockprice["close"]
idx = df_stockprice.groupby("symbol")["date"].idxmax()
df_stockprice = df_stockprice.loc[idx].reset_index(drop=True)

Loaded SILVER Layer stock price data from: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SILVER\StockPrice\StockPrice_data.csv


### Merge data and calculate PnL


In [18]:
# Merge the grouped holdings data with the latest stock price data on symbol
df_grouped = pd.merge(
    df_grouped,
    df_stockprice[["symbol", "close_price"]],
    on="symbol",
    how="left",
)

# Calculate the close amount and PnL amount for each entry
df_grouped["close_amount"] = (
    df_grouped["close_price"] * df_grouped["total_quantity"]
)
df_grouped["pnl_amount"] = (
    df_grouped["close_amount"] - df_grouped["total_amount"]
)

# Round numerical columns to 2 decimal places
df_grouped = df_grouped.round(2)

# Sort and format the DataFrame
df_grouped = df_grouped.sort_values(by=["segment", "symbol"]).reset_index(
    drop=True
)

In [19]:
# Prepare a list to hold the final result
current_holding = []
for _, row in df_grouped.iterrows():
    # Filter current holdings matching each grouped entry
    df_filtered = df_current_holding[
        (df_current_holding["scrip_name"] == row["scrip_name"])
        & (df_current_holding["symbol"] == row["symbol"])
        & (df_current_holding["exchange"] == row["exchange"])
        & (df_current_holding["segment"] == row["segment"])
    ]

    # Convert the row to a dictionary and add historical data
    row = row.to_dict()
    row["history"] = df_filtered.to_dict(orient="records")
    current_holding.append(row)

In [20]:
df_holdings = pd.read_csv(holdingshistory_gold_file_path.path)
df_holdings["date"] = pd.to_datetime(df_holdings["date"]).dt.date
print(f"Loaded GOLD Layer holdings data from: {holdingshistory_gold_file_path}")

Loaded GOLD Layer holdings data from: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\Holdings\HoldingsHistory_data.csv


In [21]:
df_holdings_trands = (
    df_holdings.groupby("date")[
        [
            "holding_amount",
            "open_amount",
            "high_amount",
            "low_amount",
            "close_amount",
        ]
    ]
    .sum()
    .reset_index()
)
df_holdings_trands = df_holdings_trands.round(2)
df_holdings_trands = df_holdings_trands.rename(
    columns={
        col: col.replace("_amount", "") for col in df_holdings_trands.columns
    }
)
df_holdings_trands = df_holdings_trands[
    [
        "date",
        "open",
        "high",
        "low",
        "close",
        "holding",
    ]
]
df_holdings_trands = df_holdings_trands.sort_values(by=["date"]).reset_index(
    drop=True
)
holdings_trands = df_holdings_trands.to_dict(orient="records")

In [22]:
print(f"Read GOLD Layer ProfitLoss data from: {profitloss_gold_file_path}")
## PRESENTATION LAYER
df = pd.read_csv(profitloss_gold_file_path.path)

# Assuming your DataFrame is named df
df["stock_name"] = df.apply(
    lambda row: (
        row["symbol"] if row["symbol"] == "NIFTY" else row["scrip_name"]
    ),
    axis=1,
)

# Ensure the columns are datetime64 dtype
df["open_datetime"] = pd.to_datetime(df["open_datetime"])
df["close_datetime"] = pd.to_datetime(df["close_datetime"])

# Calculate the difference in days
df["days"] = (df["close_datetime"] - df["open_datetime"]).dt.days

# Sort and format the DataFrame
df = df.sort_values(by=["segment", "symbol"]).reset_index(drop=True)

Read GOLD Layer ProfitLoss data from: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


In [23]:
# Grouping by 'name' and 'type'
result = []
grouped = df.groupby(["segment", "exchange", "symbol", "stock_name"])

for (segment, exchange, symbol, stock_name), group in grouped:
    group_dict = {
        "segment": segment,
        "exchange": exchange,
        "symbol": symbol,
        "stock_name": stock_name,
        "days": (
            group["close_datetime"].max() - group["open_datetime"].min()
        ).days,
        "quantity": group["quantity"].sum(),
        "avg_price": round(
            group["open_amount"].sum() / group["quantity"].sum(), 2
        ),
        "sell_price": round(
            group["close_amount"].sum() / group["quantity"].sum(), 2
        ),
        "pnl": group["pnl_amount"].sum(),
        "history": group[
            [
                "scrip_name",
                "position",
                "quantity",
                "days",
                "open_datetime",
                "open_price",
                "open_amount",
                "close_datetime",
                "close_price",
                "close_amount",
                "pnl_amount",
                "pnl_percentage",
            ]
        ].to_dict(orient="records"),
    }
    result.append(group_dict)

### Prepare final output dictionary and write to JSON file


In [24]:
# Create the final output dictionary for the API
output = {
    "financial_summary": {
        "invested_value": round(df_grouped["total_amount"].sum(), 2),
        "current_value": round(df_grouped["close_amount"].sum(), 2),
        "pnl_value": round(df_grouped["pnl_amount"].sum(), 2),
    },
    "current_holding_data": current_holding,
    "holdings_trands_data": holdings_trands,
    "profitloss_summary": {
        "invested_value": round(df["open_amount"].sum(), 2),
        "sold_value": round(df["close_amount"].sum(), 2),
        "pnl_value": round(df["pnl_amount"].sum(), 2),
    },
    "profit_loss_data": result,
}

# Write the result to a JSON file
with open(api_file_path.path, "w") as json_file:
    json.dump(output, json_file, indent=4, default=str)

print(f"Data written to {api_file_path}")

Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\API\API_data.json
